In [52]:
import numpy as np
import rsatoolbox
from rsatoolbox.util.matrix import pairwise_contrast
from scipy.spatial.distance import squareform
from scipy import io
import matplotlib.pyplot as plt
import rsatoolbox.data as rsd # abbreviation to deal with dataset
import rsatoolbox.rdm as rsr
from scipy import linalg
from scipy.optimize import minimize

In [2]:
pairs = pairwise_contrast(np.arange(3))
pairs[pairs==-1]=1
pairs

array([[1., 1., 0.],
       [1., 0., 1.],
       [0., 1., 1.]])

In [3]:
np.zeros((6-4+1,6-4+1))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [4]:
T = np.block([[np.eye(3),np.zeros((6-4+1,6-4+1))],[0.5*pairs,np.diag(-0.5*np.ones(3))]])

In [5]:
vec_G = np.array([3,4,7,-1,2,-1])

In [6]:
G = np.diag(vec_G[0:3])+squareform(vec_G[3:len(vec_G)])

In [7]:
G

array([[ 3, -1,  2],
       [-1,  4, -1],
       [ 2, -1,  7]])

In [8]:
# create a dataset object
measurements = io.matlab.loadmat('92imageData/simTruePatterns.mat')
measurements = measurements['simTruePatterns2']
nCond = measurements.shape[0]
nVox = measurements.shape[1]
# now create a  dataset object
des = {'session': 1, 'subj': 1}
obs_des = {'conds': np.array(['cond_%02d' % x for x in np.arange(nCond)])}
chn_des = {'voxels': np.array(['voxel_' + str(x) for x in np.arange(nVox)])}
data = rsd.Dataset(measurements=measurements,
                   descriptors=des,
                   obs_descriptors=obs_des,
                   channel_descriptors=chn_des)

In [9]:
data.measurements.shape

(92, 100)

In [10]:
RDM_euc = rsr.calc_rdm(data, descriptor='conds')
RDM_euc.dissimilarities

array([[1.79168033, 1.79025977, 1.76699112, ..., 1.76469464, 1.72393365,
        1.78983719]])

In [11]:
#pairs.shape

4095+92-1

4186

In [12]:
print(pairs.shape)
print(np.diag(-0.5*np.ones(92-1)).shape)

(3, 3)
(91, 91)


In [13]:
pairs = pairwise_contrast(np.arange(91))
pairs[pairs==-1]=1
pairs.shape
T = np.block([[np.eye(92-1),np.zeros((92-1,4186-92+1))],[0.5*pairs,np.diag(-0.5*np.ones(4186-92+1))]])

In [14]:
vec_G = RDM_euc.dissimilarities@np.transpose(T)

In [15]:
vec_G.shape

(1, 4186)

In [16]:
G = np.diag(vec_G[0,0:91])+squareform(vec_G[0,91:len(vec_G)])

In [17]:
np.max(linalg.eigvals(G))

(2.665013649930843+0j)

In [18]:
rdm=vec_G@linalg.inv(np.transpose(T))

In [19]:
np.sum(rdm-RDM_euc.dissimilarities)

2.886579864025407e-15

In [20]:
matlab_data = io.matlab.loadmat('rdms_inferring/modelRDMs_A2020.mat')
matlab_data = matlab_data['modelRDMs']
n_models = len(matlab_data[0])
model_names = [matlab_data[0][i][0][0] for i in range(n_models)]
measurement_model = [matlab_data[0][i][1][0] for i in range(n_models)]
rdms_array = np.array([matlab_data[0][i][3][0] for i in range(n_models)])

In [21]:
rdms_array.shape

(63, 4186)

In [22]:
model_rdms = rsatoolbox.rdm.RDMs(rdms_array,
                            rdm_descriptors={'brain_computational_model':model_names,
                                             'measurement_model':measurement_model},
                            dissimilarity_measure='Euclidean'
                           )

In [108]:
model_rdms.dissimilarities.shape

(63, 4186)

In [100]:
conv1_rdms = model_rdms.subset('brain_computational_model','conv1')
conv2_rdms = model_rdms.subset('brain_computational_model','conv2')

#fig, ax, ret_val = rsatoolbox.vis.show_rdm(conv1_rdms, rdm_descriptor='measurement_model', figsize=(10,10))

In [108]:
conv2_rdms

rsatoolbox.rdm.RDMs(
dissimilarity_measure = 
Euclidean
dissimilarities = 
[[1.83048064e+08 1.69640224e+08 1.40488128e+08 ... 2.10742496e+08
  3.26697440e+08 3.42092256e+08]
 [1.83048064e+08 1.69640224e+08 1.40488128e+08 ... 2.10742496e+08
  3.26697440e+08 3.42092256e+08]
 [2.17082656e+04 2.39821719e+04 1.27935957e+04 ... 1.72578789e+04
  3.17397969e+04 2.80723281e+04]
 ...
 [3.04157304e+03 3.47406783e+03 3.59717928e+03 ... 4.63869692e+03
  2.59170893e+04 1.88518028e+04]
 [1.82201945e+07 1.66813513e+07 1.79520879e+07 ... 2.53642351e+07
  5.94480481e+07 5.92892881e+07]
 [1.80922950e+03 2.23002314e+03 3.19083742e+03 ... 3.95388337e+03
  2.71566818e+04 1.97499778e+04]]
descriptors = 
{}
rdm_descriptors = 
{'brain_computational_model': ['conv2', 'conv2', 'conv2', 'conv2', 'conv2', 'conv2', 'conv2', 'conv2', 'conv2', 'conv2', 'conv2'], 'measurement_model': ['complete', 'smooth1sepFeat', 'smooth1rpwavgFeat', 'smooth2sepFeat', 'smooth2rpwavgFeat', 'smooth3sepFeat', 'smooth3rpwavgFeat', 'smoot

In [101]:
conv1_rdms.dissimilarities.shape

(11, 4186)

In [102]:
vector1 = conv1_rdms.get_vectors()
vector2 = conv2_rdms.get_vectors()

In [103]:
vec_G1 = vector1@np.transpose(T)
vec_G2 = vector2@np.transpose(T)

In [104]:
len(vec_G1[0])

4186

In [105]:
G1 = np.diag(vec_G1[0,0:91])+squareform(vec_G1[0,91:len(vec_G1[0])])
G2 = np.diag(vec_G2[0,0:91])+squareform(vec_G2[0,91:len(vec_G2[0])])

In [82]:
fun = lambda a: np.sqrt((np.log(linalg.eigvalsh(G1,np.exp(a)*G2))**2).sum())

In [83]:
res = minimize(fun,0, method='Nelder-Mead')

In [85]:
print(np.exp(res.x))
a_true = 1/np.exp(-np.log(linalg.eigvalsh(G1,G2)).sum()/np.shape(G1)[0])
print(a_true)


[5.2206661]
5.220808985368925


In [112]:
res.fun

5.290773680152389

In [106]:
value = np.empty((len(vec_G1), len(vec_G2)))
k1 = 0
for v1 in vec_G1:
    for v2 in vec_G2:
        G1 = np.diag(v1[0:91])+squareform(v1[91:len(v1)])
        G2 = np.diag(v2[0:91])+squareform(v2[91:len(v2)])
        value[k1, k2] = np.min(linalg.eigvalsh(G1,G2))


In [107]:
value[0]

array([ 2.31584178e+77, -1.73059806e-77,  1.01222130e-01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [116]:
len(G1)

91

In [120]:
value = np.empty((len(vec_G1), len(vec_G2)))
k1 = 0
for v1 in vec_G1:
    k2 = 0
    for v2 in vec_G2:
        #value[k1, k2] = func(v1, v2)
        G1 = np.diag(v1[0:91])+squareform(v1[91:len(v1)])
        G2 = np.diag(v2[0:91])+squareform(v2[91:len(v2)])
        #a_true = 1/np.exp(-np.log(linalg.eigvalsh(G1,G2)).sum()/np.shape(G1)[0])
        fun = lambda a: np.sqrt((np.log(linalg.eigvalsh(np.exp(a[0])*G1+np.exp(a[1])*np.eye(len(G1)),G2))**2).sum())
        res = minimize(fun,(0,0), method='Nelder-Mead')
        value[k1, k2] = res.fun
        k2 += 1
    k1 += 1

In [121]:
value

array([[ 5.29077359,  5.29077359, 12.63555846,  5.95183688, 15.67259173,
         7.02442601, 18.73029497,  8.6206581 , 20.58589213,  9.56885043,
        20.66787861],
       [ 5.29077359,  5.29077359, 12.63555846,  5.95183688, 15.67259173,
         7.02442601, 18.73029497,  8.6206581 , 20.58589213,  9.56885043,
        20.66787861],
       [ 7.54052611,  7.54052611, 11.83517869,  7.96895331, 14.84238928,
         6.87982048, 17.83653833, 10.81158573, 19.72774294, 10.15126533,
        19.88771302],
       [ 6.14584811,  6.14584811, 12.52152575,  6.22266933, 15.50504853,
         5.94330088, 18.47283904,  8.47532308, 20.17704416,  8.84375234,
        20.21469626],
       [ 4.44238692,  4.44238692, 11.7415162 ,  5.43625232, 14.70090408,
        10.34166722, 17.64715479, 12.02004047, 19.52945698, 10.19880083,
        19.70184608],
       [ 4.10575452,  4.10575452, 12.5679656 ,  7.39860415, 15.50551052,
         7.44879609, 18.39778715,  8.64082244, 19.96401162,  9.28786563,
        19.953